### <center>Youtube频道视频信息爬虫</center>

功能描述：爬取`YouTube`某个频道下的所有视频的信息，包括视频ID、标题、描述、发布时间，并将其保存为`csv`文件

依赖包安装：请确保已安装了`google-api-python-client`库。如果没有，请使用以下命令安装:
``` bash
pip install google-api-python-client
```
运行`.ipynb`文件需要`ipykernel`包,运行以下命令将`ipykernel`安装到 `Python`环境中:
``` bash 
conda install -n reptile ipykernel --update-deps --force-reinstall
```

In [13]:
import os # 导入Python内置的os模块
import csv # 导入Python内置的csv模块
from googleapiclient.discovery import build 
# 从googleapiclient包中的discovery模块导入build函数,用于访问Google API服务

#### 获取YouTube API密钥

1. 访问[Google Cloud Platform控制台](https://console.cloud.google.com/)，然后登录到您的Google帐户。

![](assets/屏幕截图-2023-05-05-144025.png)

首次登录会要求选取一个位置集中创建和管理您的`Google Cloud`实例、磁盘、网络及其他资源。

![](assets/屏幕截图-2023-05-05-144906.png)


2. 如果您已有项目，可以从项目下拉列表中选择一个。

![](assets/屏幕截图-2023-05-05-145149.png)

如果您尚未创建项目，请点击“创建项目”按钮，然后输入项目名称和其他相关信息。

![](assets/屏幕截图-2023-05-05-145338.png)


3. 在左侧导航栏中，点击“导航菜单”图标，然后选择“API和服务”>“库”。

![](assets/屏幕截图-2023-05-05-145820.png)

在搜索框中输入`YouTube Data API v3`，然后点击搜索结果中的`YouTube Data API v3`。

![](assets/屏幕截图-2023-05-05-145926.png)

![](assets/屏幕截图-2023-05-05-150024.png)


4. 点击“启用”按钮以启用`API`。

![](assets/屏幕截图-2023-05-05-150051.png)

5. `API`启用后，您将被重定向到`API`概览页面。在此页面上，点击“创建凭据”按钮。

![](assets/屏幕截图-2023-05-05-150201.png)


6. 在“创建凭据”页面上，选择“公开数据”作为凭据类型。系统将自动生成一个新的API密钥。

![](assets\屏幕截图-2023-05-05-150757.png)


7. 复制新生成的`API`密钥，并将其粘贴到`Python`代码中，替换`api_key`。

![](assets/屏幕截图-2023-05-05-150854.png)


8. 现在，您已经成功获取了`YouTube API密钥`，可以在代码中使用它来访问`YouTube Data API`。请确保保管好您的API密钥，不要与他人共享。如果需要，您可以在`Google Cloud Platform`控制台中的“API和服务”>“凭据”页面上管理和删除您的`API密钥`。

In [14]:
api_key = ""  # YouTube API密钥
# 使用Google API构建一个访问YouTube视频服务的客户端对象
youtube = build("youtube", "v3", developerKey=api_key)

In [15]:
"""
函数功能:获取指定YouTube频道的所有视频信息
传入参数:接受一个参数channel_id作为要获取视频的频道ID
返回值:返回一个包含所有视频信息的列表videos
"""
def get_channel_videos(channel_id):
    # 定义了空列表video_ids用于保存视频id和变量next_page_token用于存储下一页数据的令牌
    video_ids = []
    next_page_token = None
    # 循环获取下一页视频信息
    while True:
        # 使用googleapiclient.discovery模块的search().list()方法来向YouTube API发送请求
        request = youtube.search().list(
            part="id",  # 要获取的视频的基本信息(如标题、描述和发布时间等)
            channelId=channel_id,  # 要查询的频道ID
            maxResults=50,  # 每页最多返回的视频数
            pageToken=next_page_token,  # 要获取的下一页数据的令牌
            type="video",  # 要查询的资源类型（这里为video，即视频）
        )
        # 执行查询请求并将结果保存到response变量
        response = request.execute()
        # 将YouTube API响应中的视频ID提取出来并添加到现有的视频ID列表中
        video_ids.extend([item["id"]["videoId"] for item in response["items"]])
        # 获取YouTube API响应中的下一页令牌（next page token）的值
        next_page_token = response.get("nextPageToken")
        # 当没有下一页数据时，退出循环
        if next_page_token is None:
            break
    videos = [] # 创建空列表videos用于存储视频详细信息
    for video_id in video_ids:  # 遍历之前提取出的视频id列表
        request = youtube.videos().list( # 获取特定视频的详细信息 
            part="snippet,contentDetails,player,statistics,status", id=video_id)
        response = request.execute() # 响应包含请求的视频详细信息
        # 通过extend()方法将response中的视频信息添加到videos列表中
        videos.extend(response["items"])
        # 判断是否还有下一页数据，如果有则更新next_page_token变量，并继续循环获取下一页数据
        next_page_token = response.get("nextPageToken")
    # 返回所有视频信息的列表videos
    return videos

要想了解Youtube视频的video属性及其详细信息,可以参考 https://developers.google.com/youtube/v3/docs/videos?hl=zh-cn

![](assets/屏幕截图-2023-05-07-004356.png)

In [16]:
"""
函数功能:将指定的YouTube视频信息保存到CSV文件
传入参数:videos表示要保存的视频信息列表,csv_name表示要保存到的CSV文件名
返回值:无
"""
def save_to_csv(videos, csv_name):
    # 使用Python内置的open()方法打开指定文件
    with open(csv_name, mode="w", newline="", encoding="utf-8") as file:
        # 创建一个csv.writer对象来写入CSV格式数据
        writer = csv.writer(file)
        # 使用writerow()方法写入表头行
        writer.writerow(["id", "title", "publishedAt", "duration", "definition", "caption", "licensedContent",
                        "viewCount", "likeCount", "commentCount", "description", "embeddable", "player"])
        # 写入每个视频对应的行
        for video in videos:
            # 视频的详细信息如下
            writer.writerow([
                video["id"],  # YouTube用于唯一标识视频的ID
                video["snippet"]["title"],  # 视频标题
                video["snippet"]["publishedAt"],  # 发布日期和时间
                video["contentDetails"]["duration"],  # 视频时长
                video["contentDetails"]["definition"],  # 清晰度
                video["contentDetails"]["caption"],  # 是否有字幕
                video["contentDetails"]["licensedContent"],  # 是否受版权保护
                video["statistics"].get("viewCount", "N/A"),  # 观看次数
                video["statistics"].get("likeCount", "N/A"),  # 点赞次数
                video["statistics"].get("commentCount", "N/A"),  # 评论次数
                video["snippet"]["description"],  # 视频描述
                video["status"]["embeddable"],  # 视频是否可以嵌入到网页中以播放
                video["player"], # 视频的嵌入代码，可将其插入到网页中以播放视频
            ])

#### 获取YouTube频道ID

1. 打开您要爬取的`YouTube`频道页面。

![](assets/屏幕截图-2023-05-05-164509.png)

2. 使用 `F12` 或鼠标右键“检查(`Inspect Element`)”即可打开浏览器的调试工具

![](assets/屏幕截图-2023-05-05-152759.png)

3. 找到搜索栏或者使用快捷键`Ctrl+F`,查找"`/channel/`"。频道ID是URL中的一串字符，通常以“`/channel/`”开头，例如:

   - /channel/UCoC47do520os_4DBMEFGg4A

   在这些示例中，频道ID为“`UCoC47do520os_4DBMEFGg4A`”

![](assets/屏幕截图-2023-05-05-164639.png)


4. 将频道`ID`复制到的`Python`代码中，替换`channel_id`。

In [17]:
def run():
    # YouTube频道名称
    channel_name = "文昭思緒飛揚 - Wen Zhao Studio"
    # YouTube频道ID
    channel_id = "UCTu_hTaVf3DJMpMIyOAq2Ew"
    # 获取指定YouTube频道的所有视频信息
    videos = get_channel_videos(channel_id)
    # 将指定的YouTube视频信息保存到csv文件
    save_to_csv(videos, channel_name+".csv")
    # 输出保存文件信息的结果
    print(f"视频信息已保存到 {channel_name}.csv")

In [18]:
# 运行程序
if __name__ == "__main__":
    run()

视频信息已保存到 文昭思緒飛揚 - Wen Zhao Studio.csv


#### 解决使用Excel打开csv文件出现乱码的问题

1. 创建一个新的Excel文件

![](assets/屏幕截图-2023-05-07-002333.png)


2. 切换至 “数据” 菜单,选择数据来源为 “自文本” 选择 CSV 文件,选择导出的 CSV 文件

![](assets/屏幕截图-2023-05-07-002433.png)


3. 出现文本导入向导,文件原始格式选择 “65001：Unicode (UTF-8)” ,分割符选择“逗号”,最后点击加载

![](assets/屏幕截图-2023-05-07-002517.png)


4. 最后得到正常解码的xlsx文件

![](assets/屏幕截图-2023-05-07-002608.png)